![MONAI Logo](monai.png)

# Deploying a MedNIST Classifier App with MONAI Deploy App SDK (Prebuilt Model)

This tutorial demos the process of packaging a trained model using MONAI Deploy App SDK into an artifact that can be run as a local application that performs inference, a workflow job doing the same, and a Docker containerized workflow execution.

## Implementing and Packaging Application with MONAI Deploy App SDK

Based on the Torchscript model(`classifier.zip`), we will implement an application that processes an input Jpeg image and write the prediction(classification) result as a JSON file(`output.json`).

In our inference application, we will define two operators:

1. `LoadPILOperator` - Load a JPEG image from the input path and pass the loaded image object to the next operator.
    - This Operator does similar job with `LoadImage(image_only=True)` transform in *train_transforms*, but handles only one image.
    - **Input**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
2. `MedNISTClassifierOperator` - Pre-transform the given image by using MONAI's `Compose` class, feed to the Torchscript model (`classifier.zip`), and write the prediction into a JSON file(`output.json`)
    - Pre-transforms consist of three transforms -- `AddChannel`, `ScaleIntensity`, and `EnsureType`.
    - **Input**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: a folder path that the prediction result(`output.json`) would be written ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))

The workflow of the application would look like this.

<img src="images/mednist_workflow.png" alt="Workflow" style="width: 600px;margin-left:auto;margin-right:auto;"/>

### Contents
* [Setup](#setup)
* [Creating Operator Class](#create_op)
* [Creating Application Class](#create_app)
* [Executing App Locally](#execute_app_local)
* [Conclusion](#conclusion)




### Using Google Colab

This notebook has the pip command for installing MONAI and will be added to any subsequent notebook.

**Enabling GPU Support**

To use GPU resources through Colab, change the runtime to GPU:

1. From the **"Runtime"** menu select **"Change Runtime Type"**
2. Choose **"GPU"** from the drop-down menu
3. Click **"SAVE"**

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not)

### Verify GPU Access

Running **!nvidia-smi** in a cell will verify this has worked and show you what kind of hardware you have access to.    

In [ ]:
# Install necessary image loading/processing packages for the application
!pip install -q "Pillow"
!pip install -q "scikit-image"
!pip install -q "wget"
!pip install -q "pydicom"
!pip install -q "highdicom"
!pip install -q "matplotlib"
!pip install -q "typeguard==2.12.1"
%matplotlib inline

# Install MONAI Deploy App SDK package
!pip install -qU "monai-deploy-app-sdk"
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==1.1.0"

### 1. Setup

To begin, check that the NVIDIA driver has been installed correctly. The `nvidia-smi` command should run and output information about the GPUs on your system:

In [ ]:
!nvidia-smi

### 1.1 Setup environment

We'll set up folder called notebook_0 where we'll extract our data, models, and write out output.

In [ ]:
NOTEBOOK_ROOT="notebook_0/"
!mkdir -p notebook_0
!gdown "https://drive.google.com/uc?id=1yJ4P-xMNEfN6lIOq_u6x1eMAq1_MJu-E"
!unzip -o mednist_classifier_data.zip -d notebook_0

### 1.2 Setup imports

Let's import necessary classes/decorators and define `MEDNIST_CLASSES`.

In [ ]:
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]

from monai.deploy.operators.dicom_text_sr_writer_operator import DICOMTextSRWriterOperator, EquipmentInfo, ModelInfo
from typing import Text

<a id='create_op'></a>
## 2. Creating Operator classes

### 2.1 LoadPIL Operator

In [ ]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        op_output.set(output_image)

### 1.2 MedNIST Classifier Operator

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.output("result_text", Text, IOType.IN_MEMORY)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # get the class name
        print(result)
        op_output.set(result, "result_text")

        ##### Get output (folder) path and create the folder if not exists
        # output_folder = op_output.get().path
        # output_folder.mkdir(parents=True, exist_ok=True)

        ### The following gets the App context's output path, instead the operator's.
        output_folder = context.output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)
        
        
        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)

<a id='create_app'></a>
## 2. Creating Application class

Our application class would look like below.

It defines `App` class inheriting `Application` class.

`LoadPILOperator` is connected to `MedNISTClassifierOperator` by using `self.add_flow()` in `compose()` method of `App`.

In [ ]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)

<a id='execute_app_local'></a>
## 3. Executing app locally

We can execute the app in the Jupyter notebook.

In [ ]:
app = App()
app.run(input=NOTEBOOK_ROOT+"/input/AbdomenCT_007000.jpeg", output=NOTEBOOK_ROOT+"/output", model=NOTEBOOK_ROOT+"classifier.zip")

In [ ]:
!cat $NOTEBOOK_ROOT/output/output.json

<a id='conclusion'></a>
## Conclusion

In this notebook, we have walked through the process to create a classification task using a pre-built model using MONAI Deploy App SDK and running the application in Jupyter.